In [4]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np

data = pd.read_csv("train_data.csv",sep=",")
test= pd.read_csv("test_data.csv",sep=",")
test_target= pd.read_csv("sample_submission.csv",sep=",")

data['Work load Average/day '] = [x.replace(',', '') for x in data['Work load Average/day ']]
data['Work load Average/day '] = data['Work load Average/day '].astype(int)

target=data['Absent']
data=data.drop(['Absent'],1)
test_target=test_target['Absent']
data=data.drop(['ID'],1)
test=test.drop(['ID'],1)

#selectkbest
selector = SelectKBest(chi2, k=2) #testar de 1 a 19
fit = selector.fit(data, target)
#features selection
cols = selector.get_support(indices=True)
cols_names = list(data.columns[cols])
data = data[cols_names]
test = test[cols_names]

In [7]:
normalizer = preprocessing.Normalizer(norm='l2') 
values_normalized = normalizer.transform(data.values)
data = pd.DataFrame(values_normalized, columns=data.columns)
values_normalized = normalizer.transform(test.values)
test = pd.DataFrame(values_normalized, columns=test.columns)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                   {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [8]:
print("# Tuning hyper-parameters for precision")    
clf = GridSearchCV(SVC(), tuned_parameters, cv=2, scoring='precision')
clf.fit(data, target)

print("Best parameters set found on:")
print(clf.best_params_)
print()
parametros = clf.best_params_
    
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

print()
new = clf.predict(test)
y_true, y_pred = test_target, clf.predict(test)

print(classification_report(y_true, y_pred, labels=np.unique(y_pred)))
    
print()
    
print("# Tuning hyper-parameters for recall")    
clf = GridSearchCV(SVC(), tuned_parameters, cv=2, scoring='recall')
clf.fit(data, target)

print("Best parameters set found on:")
print(clf.best_params_)
print()
parametros = clf.best_params_
    
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

print()
new = clf.predict(test)
y_true, y_pred = test_target, clf.predict(test)

print(classification_report(y_true, y_pred, labels=np.unique(y_pred)))

# Tuning hyper-parameters for precision
Best parameters set found on:
{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

0.790 (+/-0.002) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.790 (+/-0.002) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.790 (+/-0.002) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.790 (+/-0.002) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.790 (+/-0.002) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.790 (+/-0.002) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.790 (+/-0.002) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.790 (+/-0.002) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.790 (+/-0.002) for {'C': 1, 'kernel': 'linear'}
0.790 (+/-0.002) for {'C': 10, 'kernel': 'linear'}
0.790 (+/-0.002) for {'C': 100, 'kernel': 'linear'}
0.790 (+/-0.002) for {'C': 1000, 'kernel': 'linear'}

              precision    recall  f1-score   support

           1       0.82      1.00      0.90       196

   micro avg       0.82      1.00      0.90       